In [1]:
# start here
text = open('wiki_all.txt', 'r').read()

In [2]:
# Hyperparameters
batch_size = 32 # how many independent sequences we will process in parallel 
block_size = 8 # what is the maximum context length for predictions?
eval_iters = 10
eval_interval = 300
learning_rate = 3e-4
max_iters = 3000

In [3]:
# generate embedings
import tiktoken
#encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.get_encoding("r50k_base")
text_encoded = encoding.encode(text)

from tinygrad import Tensor, nn, dtypes, TinyJit, Context
import numpy as np
data = Tensor(text_encoded, dtype=dtypes.long)
# split into training and validation
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
#print(len(data))

In [14]:
Tensor.manual_seed(1337)

def get_batch(split):
    # generate small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = Tensor.randint((batch_size,), high=(len(data) - block_size)).numpy()
    x = Tensor([data.numpy()[i:i+block_size] for i in ix], dtype=dtypes.long)
    y = Tensor([data.numpy()[i+1:i+block_size+1] for i in ix], dtype=dtypes.long)
    return x, y

def get_batch_2(split):
    data = train_data if split == 'train' else val_data
    #rand = Tensor.randint(high=(len(data) - block_size))
    rand = np.random.randint(0,(len(data)-block_size))
    #rand = 1337

    x = data[rand:rand+block_size*batch_size].view(batch_size, block_size)
    y = data[rand+1:rand+block_size*batch_size+1].view(batch_size, block_size)

    return x, y

In [5]:
class BigramLanguageModel:
    def __init__(self, vocab_size, embed_size):
        self.token_embedding_table = nn.Embedding(vocab_size=vocab_size, embed_size=embed_size)

    def __call__(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            loss = logits.cross_entropy(targets)
            print(loss.item())
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = logits.softmax(axis=1)
            idx_next = probs.multinomial(num_samples=1)
            idx = idx.cat(idx_next, dim=1)
        return idx
        
m = BigramLanguageModel(encoding.n_vocab, 500) # vocab size via tiktoken

In [12]:
@TinyJit
def estimate_loss():
    out = {}
    Tensor.no_grad, Tensor.training = True, False
    for split in ['train', 'val']:
        losses = Tensor.zeros(eval_iters, requires_grad=False).contiguous()
        for k in range(eval_iters):
            X, Y = get_batch_2(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    Tensor.no_grad, Tensor.training = False, True
    return out

In [7]:
# create optomizer 
optimizer = nn.optim.AdamW(nn.state.get_parameters(m), lr=learning_rate)

In [15]:
# training
import time

@TinyJit
def step(xb, yb):
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()
    
with Tensor.train():
    for steps in range(max_iters):
        if steps % eval_interval == 0:
            t0 = time.time()
            losses = estimate_loss()
            t1 = time.time()
            print(t1-t0)
            print(f"step {steps}: train loss {losses['train'].item():.4f}, val loss {losses['val'].item():.4f}")
        
        # sample data
        xb, yb = get_batch_2('train')
    
        # evaluate loss
        loss = step(xb, yb)
    
        #print(steps, ':', loss)
        
print("show me the booty:")
print(encoding.decode(m.generate(Tensor.zeros((1,1), dtype=dtypes.long), max_new_tokens=100)[0].tolist()))

2.355067253112793
2.889179229736328
2.5980112552642822
2.6221630573272705
3.082728862762451
2.694859027862549
2.8649260997772217
2.767436981201172
2.8399858474731445
2.598045825958252
2.7191879749298096
4.322714805603027
2.3058266639709473
2.7919766902923584
2.670599937438965
2.524890899658203
2.743241786956787
2.5733513832092285
2.7432801723480225
3.0345325469970703
1.6538870334625244
step 0: train loss 2.7312, val loss 2.8430


AssertionError: duplicate inputs to JIT